# Project Natural Computing

First we add some imports that are needed

In [1]:
import numpy as np
from copy import deepcopy
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

Next, we define the activation functions. There is a choice between ReLU, Sigmoid and SoftMax

In [2]:
def ReLU(mat, gradient = False):
    if gradient:
        return mat * (mat > 0), 1 * (mat > 0)
    return mat * (mat > 0)

def Sigmoid(mat, gradient = False):
    sigm = 1/(1 + np.exp(-mat))
    if gradient:
        return sigm, sigm * (1 - sigm)
    return sigm

def SoftMax(mat, gradient = False):
    exp_mat = np.exp(mat)
    res_mat =  exp_mat/np.sum(exp_mat, axis = 1, keepdims = True)
    if gradient:
        return res_mat, res_mat * (1 - res_mat)
    return res_mat

Next, we have a network class which contains the number of layer, activation function, weights, biases and current fitness.

The forward function pushes the samples through the current network and returns the return values.

In [3]:
class Network:
    def __init__(self, node_seq, activation_functions, Xavier = True):
        self.node_seq = node_seq
        self.num_layers = len(node_seq) - 1
        self.activation = activation_functions
        self.weight_mats = []
        self.bias_vecs = []
        self.fitness = 0
        self.weight_mats_gradient = []
        self.bias_vecs_gradient = []
        for idx in range(self.num_layers):
            n_in = node_seq[idx]
            n_out = node_seq[idx + 1]
              # use Xavier initialization
            bound = np.sqrt(6/(n_in + n_out))
            if Xavier:
                self.weight_mats.append(np.random.uniform(-bound, bound, size = (n_in, n_out)))
            else: 
                self.weight_mats.append(np.zeros((n_in, n_out)))
            self.bias_vecs.append(np.zeros(n_out))
            self.weight_mats_gradient.append(np.zeros((n_in, n_out)))
            self.bias_vecs_gradient.append(np.zeros(n_out))
            
            
    def forward(self, samples, gradient = False):
        # note: give inputs in the form (samples, features)
        if not gradient:
            for idx in range(self.num_layers):
                samples = self.activation[idx](samples @ self.weight_mats[idx] + self.bias_vecs[idx])
            return samples
    
        n_samples = samples.shape[0]
        sample_list = [samples]
        current_sample = samples
        gradient_list = []
        layer_gradient = []
          
          # forward step
        for idx in range(self.num_layers):
            current_sample, current_gradient = self.activation[idx](current_sample @ 
                     self.weight_mats[idx] + self.bias_vecs[idx], gradient = True)
            gradient_list.append(current_gradient)
            sample_list.append(current_sample)
            n_in = self.node_seq[idx]
            n_out = self.node_seq[idx + 1]
            layer_gradient.append(np.zeros((n_samples, n_in)))
        
          # backward step
        layer_gradient.append(2*sample_list[self.num_layers])
        for idx in range(self.num_layers - 1, -1, -1):
            self.weight_mats_gradient[idx] = sample_list[idx].T @ layer_gradient[idx + 1]
            self.bias_vecs_gradient[idx] = np.ones(n_samples).T @ layer_gradient[idx + 1]
            layer_gradient[idx] = (layer_gradient[idx + 1] * gradient_list[idx]) @ self.weight_mats[idx].T 
        
        return current_sample
        
        
net = Network([2,2,2], [ReLU, SoftMax])
net.forward(np.array([[0,0], [0,1], [1,0], [1,1]]), True)

array([[0.5       , 0.5       ],
       [0.25283717, 0.74716283],
       [0.32648448, 0.67351552],
       [0.14092044, 0.85907956]])

The next block defines several fitness functions

In [4]:
def cross_entropy_fitness(correct, predict):
    return max(np.sum(np.log(predict[range(predict.shape[0]), correct])) + predict.shape[0], 0)

  # computes fitness based on xor of all binary sequences of a given length
def xor_fitness(net, inputs, outputs, gradient = False):
    predict = net.forward(inputs, gradient)
    return cross_entropy_fitness(outputs, predict)

def xor_all_fitness(net, inputs, outputs, gradient = False):
    predict = net.forward(inputs, gradient)
    return np.log(predict[range(predict.shape[0]), outputs])

In this section, we define crossover and mutation operators

In [33]:
def crossover_minimax(net1, net2):
    node_seq = net1.node_seq
    activation = net1.activation
    new_net1 = Network(node_seq, activation, Xavier = False)
    new_net2 = Network(node_seq, activation, Xavier = False)
    for idx in range(net1.num_layers): #perhaps random choice, instead of one net all minima and one net all maxima
        new_net1.weight_mats[idx] = np.minimum(net1.weight_mats[idx], net2.weight_mats[idx])
        new_net2.weight_mats[idx] = np.maximum(net1.weight_mats[idx], net2.weight_mats[idx])
        new_net1.bias_vecs[idx] = np.minimum(net1.bias_vecs[idx], net2.bias_vecs[idx])
        new_net2.bias_vecs[idx] = np.maximum(net1.bias_vecs[idx], net2.bias_vecs[idx])
    return new_net1, new_net2
    
def crossover_layer(net1, net2):
    node_seq = net1.node_seq
    activation = net1.activation
    new_net1 = Network(node_seq, activation, Xavier = False)
    new_net2 = Network(node_seq, activation, Xavier = False)
    select_layers = np.random.uniform(size = net1.num_layers)
    for idx in range(net1.num_layers): 
        if select_layers[idx] < 0.5:
            new_net1.weight_mats[idx] = net1.weight_mats[idx]
            new_net2.weight_mats[idx] = net2.weight_mats[idx]
            new_net1.bias_vecs[idx] = net1.bias_vecs[idx]
            new_net2.bias_vecs[idx] = net2.bias_vecs[idx]
        else:
            new_net1.weight_mats[idx] = net2.weight_mats[idx]
            new_net2.weight_mats[idx] = net1.weight_mats[idx]
            new_net1.bias_vecs[idx] = net2.bias_vecs[idx]
            new_net2.bias_vecs[idx] = net1.bias_vecs[idx]
    return new_net1, new_net2
    
def mut_layer(net, gradient = False):
    mut_size = np.random.uniform()
    select_layers = np.random.uniform(size = net.num_layers)
    for idx in range(net.num_layers): 
        if select_layers[idx] < 0.5:
            n_in = net.node_seq[idx]
            n_out = net.node_seq[idx + 1]
            bound = np.sqrt(6/(n_in + n_out)) * mut_size
            if gradient:
                net.weight_mats[idx] += np.random.uniform(-mut_size, mut_size, \
                     size = (n_in, n_out)) / np.sqrt(np.maximum(np.abs(net.weight_mats_gradient[idx]), 0.01))
                net.bias_vecs[idx] += np.random.uniform(-mut_size, mut_size, \
                     size = n_out) /  np.sqrt(np.maximum(np.abs(net.bias_vecs_gradient[idx]), 0.01))
            else:
                bound = np.sqrt(6/(n_in + n_out)) * mut_size
                net.weight_mats[idx] += np.random.uniform(-bound, bound, size = (n_in, n_out))
                net.bias_vecs[idx] += np.random.uniform(-bound, bound, size = n_out)

def mut_index(net, gradient = False):
    mut_size = np.random.uniform()
    idx = int(np.random.choice(net.num_layers, 1))
    n_in = net.node_seq[idx]
    n_out = net.node_seq[idx + 1]
    idx2 = int(np.random.choice(n_in, 1))
    idx3 = int(np.random.choice(n_out, 1))
    if gradient:
        net.weight_mats[idx][idx2][idx3] += np.random.uniform(-mut_size, mut_size, \
            size = 1) / np.sqrt(np.maximum(np.abs(net.weight_mats_gradient[idx][idx2][idx3]), 0.01))
    else:
        bound = np.sqrt(6/(n_in + n_out)) * mut_size
        net.weight_mats[idx][idx2][idx3] += np.random.uniform(-bound, bound, size = 1)


The class Genetic defines the neuroevolution algorithm

In [34]:
class Genetic:
    def __init__(self, pop_size, node_seq, activation_functions, fitness_function, 
                 inp, outp, mutation_functions, mutation_probabilities, 
                 crossover_functions, crossover_probabilities, elite, gradient = False):
        self.net_list = []
        self.pop_size = pop_size
        for idx in range(pop_size):
            self.net_list.append(Network(node_seq, activation_functions))
        
        self.fitness_func = fitness_function
        self.inp = inp
        self.outp = outp
        self.grad = gradient
        self.mutation_func = mutation_functions
        self.mutation_threshold = np.cumsum(mutation_probabilities)
        self.crossover_func = crossover_functions
        self.crossover_threshold = np.cumsum(crossover_probabilities)
        self.elite = elite
        self.current_best_fitness = 0
        self.hist = []
        
    def next_generation(self):
        new_net_idx = np.random.permutation(self.pop_size)
        new_net_list = [deepcopy(self.net_list[idx]) for idx in new_net_idx]
        
        select_mutate = np.random.uniform(size=self.pop_size)
        for idx in range(self.pop_size):
            for mut_idx in range(len(self.mutation_func)):
                  # select mutation mut_idx if self.mutation_threshold[mut_idx-1] <
                  # select_mutate[idx] < self.mutation_threshold[mut_idx]
                if select_mutate[idx] < self.mutation_threshold[mut_idx]:
                    self.mutation_func[mut_idx](new_net_list[idx], self.grad)
                    break 
                    
        cross_net_idx = np.random.permutation(self.pop_size)
        cross_net_list = [deepcopy(self.net_list[idx]) for idx in cross_net_idx]
        select_cross = np.random.uniform(size = self.pop_size)
        for idx in range(self.pop_size//2):
            for cross_idx in range(len(self.crossover_func)):
                if select_cross[idx] < self.crossover_threshold[cross_idx]:
                    cross_net_list[2*idx], cross_net_list[2*idx + 1] =  \
                      self.crossover_func[cross_idx](cross_net_list[2*idx], \
                                                     cross_net_list[2*idx + 1])
                    
        new_net_list.extend(cross_net_list)
        new_net_list.extend(self.net_list)
        for net in new_net_list:
            net.fitness = self.fitness_func(net, self.inp, self.outp, self.grad)
        
        new_net_list.sort(key = lambda x: -x.fitness)
        fitnesses = [net.fitness for net in new_net_list]
        probabilities = fitnesses / np.sum(fitnesses)
        net_idx = np.random.choice(len(new_net_list), self.pop_size - self.elite, p = probabilities)
        self.net_list = [new_net_list[idx] for idx in net_idx]
        self.net_list.extend(new_net_list[:self.elite])

        self.net_list.sort(key = lambda x: -x.fitness)
        if self.net_list[0].fitness > 0.0001 + self.current_best_fitness:
            self.current_best_fitness = self.net_list[0].fitness
        self.hist.append(self.net_list[0].fitness)

# Testing XOR

In [36]:
seq_lengths = [2, 4]
all_hists = []
for seq_len in seq_lengths:
    inputs = np.array([[(num >> idx) % 2 for idx in range(seq_len)] for num in range(2**seq_len)])
    outputs = np.array([sum(seq) % 2 for seq in inputs])


    genetic_algo = Genetic(50, [seq_len, 20, 2], [ReLU, SoftMax], xor_fitness, 
                            inputs, outputs,
                            [mut_layer, mut_index], [0.5, 0.5], 
                            [crossover_minimax, crossover_layer], [0.5, 0.5], 1, True)
    for gen in range(150): #was 1500
        genetic_algo.next_generation()
    opt_net = genetic_algo.net_list[0]
    xor_all_fitness(opt_net, inputs, outputs)
    fitnesses = genetic_algo.hist
    all_hists.append(fitnesses)

#opt_net.forward([[0,0,0,0]])

In [ ]:
seq_len = 5
inputs = np.array([[(num >> idx) % 2 for idx in range(seq_len)] for num in range(2**seq_len)])
outputs = np.array([sum(seq) % 2 for seq in inputs])

genetic_algo = Genetic(50, [seq_len, 20, 2], [ReLU, SoftMax], xor_fitness, 
                        inputs, outputs,
                        [mut_layer, mut_index], [0.5, 0.5], 
                        [crossover_minimax, crossover_layer], [0.5, 0.5], 1, True)
for gen in range(150): #was 6000
    genetic_algo.next_generation()
opt_net = genetic_algo.net_list[0]
xor_all_fitness(opt_net, inputs, outputs)
fitnesses = genetic_algo.hist
all_hists.append(fitnesses)

In [ ]:
seq_lengths.append(5)
for i in range(0, len(seq_lengths)):
    plt.plot(range(0,len(all_hists[i])), all_hists[i], label = "Seq_len = " + str(seq_lengths[i]))
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend()
plt.title("Performance of the neuroevolution algorithm on different sequence lengths.")
plt.show()

# IRIS

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
#from sklearn.datasets import fetch_mldata

In [ ]:
iris = datasets.load_iris()
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.33, random_state=42)
print(np.shape(x_train))
print(np.shape(x_test))

In [ ]:
genetic_algo = Genetic(50, [4, 20, 3], [ReLU, SoftMax], xor_fitness, 
                        x_train, y_train,
                        [mut_layer, mut_index], [0.5, 0.5], 
                        [crossover_minimax, crossover_layer], [0.5, 0.5], 1)
for gen in range(200):
    genetic_algo.next_generation()
opt_net = genetic_algo.net_list[0]
xor_all_fitness(opt_net, x_train, y_train)
fitnesses = genetic_algo.hist

In [ ]:
plt.plot(range(0,len(fitnesses)), fitnesses, label = "IRIS")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend()
plt.title("Performance of the neuroevolution algorithm on iris dataset.")
plt.show()

In [ ]:
predict = opt_net.forward(x_test)
predicted = [np.argmax(x) for x in predict]
acc = metrics.accuracy_score(y_test, predicted)
print(acc)

With about 10 generations, we found an accuracy of around 80%. When the number of generations is increased to 200, we can almost always reach an accuracy > 95%.

# MNIST

In [ ]:
from sklearn.datasets import load_digits

In [ ]:
mnist = load_digits()

In [ ]:
plt.imshow(mnist.images[0]);

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.33, random_state=42)

print(np.shape(x_train))
print(np.shape(x_test))

In [ ]:
genetic_algo = Genetic(50, [64, 20, 10], [ReLU, SoftMax], xor_fitness, 
                        x_train, y_train,
                        [mut_layer, mut_index], [0.5, 0.5], 
                        [crossover_minimax, crossover_layer], [0.5, 0.5], 1)
for gen in range(10):
    genetic_algo.next_generation()
opt_net = genetic_algo.net_list[0]
xor_all_fitness(opt_net, x_train, y_train)
fitnesses = genetic_algo.hist

In [ ]:
plt.plot(range(0,len(fitnesses)), fitnesses, label = "IRIS")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend()
plt.title("Performance of the neuroevolution algorithm on iris dataset.")
plt.show()

In [ ]:
predict = opt_net.forward(x_test)
predicted = [np.argmax(x) for x in predict]
acc = metrics.accuracy_score(y_test, predicted)
print(acc)